In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from collections import Counter
import string

# Tokenization and PoS Tagging

In [ ]:
from nltk.corpus import movie_reviews

docs = []
docs_tags = []
for j, category in enumerate(movie_reviews.categories()):
    for i, fileid in enumerate(movie_reviews.fileids(category)):
        movie_file = movie_reviews.raw(movie_reviews.fileids()[i + 1000 * j])
        if (i + 1000 * j) % 200 == 0:
            print("DONE ", (i + 1000 * j)/20)
        
        doc = []
        doc_tags = []
        lower = movie_file.lower()
        words = word_tokenize(lower)
        tagged_doc = pos_tag(words)
        
        words = []
        tags = []
        for pair in tagged_doc:
            word = pair[0]
            tag = pair[1]
            words.append(word)
            tags.append(tag)
            
        doc.append(words)
        doc.append(category)
        
        docs.append(doc)
        docs_tags.append(tags)

DONE  0.0
DONE  10.0
DONE  20.0
DONE  30.0
DONE  40.0
DONE  50.0
DONE  60.0
DONE  70.0
DONE  80.0
DONE  90.0


In [2]:
# docs[0][0]

# Lemmatization

In [36]:
def wordnet_to_lemmatizer(pos_tag):
    if pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

lem = WordNetLemmatizer()

for i, doc in enumerate(docs):
    if (i % 200) == 0:
        print("DONE ", i/20)
    lem_doc = []
    
    lem_words = []
    for j, word in enumerate(doc[0]):
        tag = wordnet_to_lemmatizer(docs_tags[i][j])
        
        if tag != '':
            lemmatized_word = lem.lemmatize(word, pos=tag)
        else:
            lemmatized_word = word
            
        lem_words.append(lemmatized_word)
    lem_doc.append(lem_words)
    lem_doc.append(doc[1])
    
    docs[i] = lem_doc

DONE  0.0
DONE  10.0
DONE  20.0
DONE  30.0
DONE  40.0
DONE  50.0
DONE  60.0
DONE  70.0
DONE  80.0
DONE  90.0


In [3]:
# docs[1000]

# Negation Handling

In [41]:
negatives = set(["not", "n't", "never", "neither", "nor"])
punctuations = set([".", ",", "!", "?", ";", ":", "(", ")"])
def negation_handler(words):
    replacer = False
    for i, word in enumerate(words):
        if word in punctuations:
            replacer = False
            
        if replacer == True:
            not_word = "NOT_" + word
            words[i] = not_word
            
        if word in negatives:
            replacer = True
            
for i, doc in enumerate(docs):
    if (i % 200 == 0):
        print("DONE ", i/20)
    logical_doc = []
    
    words = doc[0]
    negation_handler(words)
    logical_doc.append(words)
    logical_doc.append(doc[1])
    
    docs[i] = logical_doc

DONE  0.0
DONE  10.0
DONE  20.0
DONE  30.0
DONE  40.0
DONE  50.0
DONE  60.0
DONE  70.0
DONE  80.0
DONE  90.0


In [4]:
# docs[1000]

# Removing Stop Words and Noise

In [ ]:
stops = list(stopwords.words('english'))
stops.append("'s")
stops.append("n't")
for i, word in enumerate(stops):
    not_word = "NOT_" + word
    stops.append(not_word)
stops = set(stops)

noise_tags = set(["CC", "CD", "DT", "EX", "FW", "IN", "LS", "PDT", "POS",
                  "PRP", "PRP$", "RP", "TO", "UH", "WDT", "WP", "WP$", "WRB",
                  ",", ":", ";", ".", '"', "'", "/", "\\", "(", ")", "@", "&", "``",
                  "'s"])
def removeWord(posTag):
    if posTag[1] in noise_tags:
        return True
    else:
        return False

for i, doc in enumerate(docs):
    if (i % 200) == 0:
        print("DONE ", i/20)
        
    filtered_doc = []
    
    filtered_words = []
    for j, word in enumerate(doc[0]):
        tag = docs_tags[i][j]
        if (word not in stops and tag not in noise_tags):
            filtered_words.append(word)
    
    filtered_doc.append(filtered_words)
    filtered_doc